In [ ]:
# using Pkg
# Pkg.activate("/home/jek354/research/ML-signproblem")
# Pkg.update()

using Lattices
using LinearAlgebra
using Combinatorics
using SparseArrays
using Plots
import Graphs
using LaTeXStrings
using Statistics
using Random
using Zygote
using Optimization, OptimizationOptimisers
using JSON
using OptimizationOptimJL
using JLD2
using KrylovKit
# using ExponentialUtilities


include("ed_objects.jl")
include("ed_functions.jl")
include("ed_optimization.jl")
include("utility_functions.jl")
# include("adiabatic_analysis.jl")

merge_jld2_folder

In [ ]:
# using KrylovKit


t = 1.0
U = 6
μ = 0  # positive incentivises fewer particles (one electron costs this much energy)
N_up = 4
N_down = 4
N =  6
half_filling = false
# lattice = Chain(6, Periodic())
lattice_dimension = (4,2)
bc = "periodic"
# lattice = Chain(6, Periodic())
lattice = Square(lattice_dimension, if bc == "periodic" Periodic() else Open() end)
# lattice = Graphs.cycle_graph(3)

models = []

reference_index = 2
# t_values = []# Float64[1,1], Float64[1,0.1], 1.0]
# for _t in t_values
#     # println(_t)
#     push!(models,HubbardModel(_t,0.0001,μ,half_filling))
# end
# U_values = [0.00001,0.01]
U_values = [0.00001; LinRange(2.1,9,20)]
U_values = sort([U_values; 10.0 .^LinRange(-3,2,40)])

for U in U_values
    # println(t)
    push!(models,HubbardModel(t,U,μ,half_filling))
end

subspace = HubbardSubspace(N, lattice)
# subspace = HubbardSubspace(N_up, N_down, lattice)

# ops = []
# if subspace.N >= 1
#     push!(ops,Matrix(create_operator(subspace,:Sx)))
# end
# push!(ops, Matrix(create_operator(subspace,:S2)))
# # op3 = Matrix(create_operator(subspace,:L2))
# push!(ops, Matrix(create_operator(subspace,:T, kind=1)))
# push!(ops, Matrix(create_operator(subspace,:T, kind=2)))
# push!(ops, Matrix(create_operator(subspace,:σ, kind=1)))
E = []
H = []
V = []
# for model ∈ models
#     push!(H, Matrix(create_Hubbard(model, subspace; perturbations=false)))
#     e, v = eigen(H[end])
#     push!(E, e)
#     push!(V, v)
# end

H = create_Hubbard(models[20], subspace; perturbations=false)

# eigsolve(H, normalize(rand(size(H)[1])), 1,:SR)

# for i ∈ 1:2
#     println(degeneracy_count(E[i]))
# end

# println(sum(abs.(h1*h2-h2*h1)))

# ops_tmp = [H[1], ops...]
# for i in eachindex(ops_tmp)
#     for j in i+1:length(ops_tmp)
#         println("($i,$j) ", sum(abs.(ops_tmp[i]*ops_tmp[j] - ops_tmp[j]*ops_tmp[i])))
#     end
# end

# H1 = create_Hubbard(model, subspace; perturbations=false)
# E1 = diag(V1'*Matrix(H1)*V1)

# @time degen_rm_U = create_consistent_basis(H, ops;reference_index=reference_index)

# dim = get_subspace_dimension(subspace)
# indexer = CombinationIndexer(reduce(vcat,collect(sites(subspace.lattice))), get_subspace_info(subspace)...)
# difference_dict = collect_all_conf_differences(indexer)


In [ ]:
function find_reprentatives(dim::Int, eig_indices::Vector{Int}, n_eigs::Vector{Int}, 
        mapping::Vector, mapping_sign::Vector)
    # 1<= eig_idx <= n_eigs
    checked_indices = Array{Any}(undef, dim)
    representative_indices = []
    associated_representative = zeros(Int, dim)
    representative_indices_inverse = zeros(Int, dim)
    magnitude = []
    periods = [[] for _ in eachindex(eig_indices)]
    for i = 1:dim
        if isassigned(checked_indices, i)
            continue
        end
        # println(i)

        period = ones(Int, length(eig_indices))
        overall_signs = ones(Int, length(eig_indices))

        # finding periods
        for (l, (map_l, sign_l)) in enumerate(zip(mapping, mapping_sign))
            j = map_l[i]
            overall_signs[l] *= sign_l[j]
            while j != i
                period[l] += 1
                j = map_l[j]
                overall_signs[l] *= sign_l[j]
            end
        end

        # searching through remaining states corresponding to representative
        index_matrix = zeros(Int, period...)
        index_matrix[ones(Int, period...)...] = i
        checked_indices[i] = Tuple(ones(Int, length(period)))
        # println(checked_indices[1:20])
        for indices in Iterators.product([1:k for k in period]...)
            if index_matrix[indices...] != 0
                continue
            end

            prev_indices = collect(indices)
            op_k = 0
            for k in eachindex(indices)
                if indices[k] > 1
                    prev_indices[k] -= 1
                    op_k = k
                    break
                end
            end

            j = mapping[op_k][index_matrix[prev_indices...]]
            index_matrix[indices...] = j
            if !isassigned(checked_indices,j)
                checked_indices[j] = indices
                associated_representative[j] = i
            end
        end

        # computing representative weight
        # mR/N + 1/2 = Z if overall_sign == -1
            # mR + N/2 = Z*N
        # mR/N = Z if overall_sign == 1
        mag = 1
        for (l, (s, p, n_eig, eig_i)) in enumerate(zip(overall_signs,period, n_eigs, eig_indices))
            if p == n_eig 
                continue
            elseif s == -1 && n_eig % (2*p) != 0
                mag *= sum(exp(1im*l*2π*((eig_i-1)*p/n_eig + 0.5)) for l=0:(n_eig ÷ p-1))
            elseif ((eig_i - 1)* p + (s == -1)*n_eig/2) % n_eig ≈ 0
                mag *= n_eig/p
            else
                mag *= 0
                break
            end
        end
        # println(mag)
        if !(mag ≈ 0)
            push!(representative_indices, i)
            representative_indices_inverse[i] = length(representative_indices)
            for (i,p) in enumerate(period)
                push!(periods[i], p)
            end
            push!(magnitude, mag)
        end

        # if i > 10
        #     break
        # end

    end
     

    return checked_indices, representative_indices, associated_representative, representative_indices_inverse, magnitude, periods
end

function construct_hubbard_hamiltonian(U::Float64, args...)
    
end
mapping = []
s_mapping = [] 
for kind in 1:2
    op = create_operator(subspace,:T, kind=kind)
    r, c, v = findnz(op)
    push!(mapping, r[c])
    push!(s_mapping, v[c])
end
checked_indices, representative_indices, associated_representative, representative_indices_inverse, magnitude, periods = find_reprentatives(size(H)[1], [2,1], [4,2], mapping, s_mapping)
# mapping(2)

(Any[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)  …  (3, 2), (2, 2), (3, 2), (1, 2), (5, 2), (4, 2), (3, 2), (2, 2), (4, 2), (5, 2)], Any[], [0, 1, 1, 1, 1, 0, 0, 0, 0, 0  …  38557, 38561, 38551, 38565, 38559, 38556, 38560, 38563, 38551, 38551], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Any[], Vector{Any}[[], []])

In [14]:
x = spzeros(size(H)[1])
x[1] = 1
op * x

8008-element SparseVector{Float64, Int64} with 1 stored entry:
  [4]  =  -1.0

In [10]:
maximum(periods)

2

In [ ]:
find_symmetric_basis(ops, [1,1,1])
new_h = operator_subspace(op, bases)